In [33]:
# Importing libraries
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Layer, Add
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import he_normal, Zeros
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras.utils.generic_utils import get_custom_objects



## 1.Activation Function

In [34]:

# Creating Swish activation function
def swish(x, beta=1):
    """
    Swish activation function.

    ARGUMENTS
        * x:    input variable.
        * beta: hyperparameter of the Swish activation function.

    Returns
        * Swish activation function applied to x.
    """

    return x * sigmoid(beta * x)



The swish activation function looks as below: 

![Sigmoid](../images/swish%20activation.png )

## 2.Vectorization

In [35]:



# %% Creating vectorization layer
class Vectorize(Layer):
    """
    Layer that vectorizes the second dimension of inputs.
    A 1 dimensional tensor is a vector 
    A 2 dimensional tensor is a matrix
    Thus, generally a tensor is a multidimensional array.
    
    
    1. init initialises the class. note that kwargs is a dictionary of keyword arguments.
    2. Call is a method that changes the input tensor into a 1D tensor, and removes the NaN values from the tensor.
    3.compute_output_shape returns the shape of the output tensor as a tuple.
    """


  
    def __init__(self, **kwargs):
        super(Vectorize, self).__init__(**kwargs)
        self.dim1 = None

    def call(self, x):
        
        #returns a true-false matrix of the same shape as x, where True is where x is NaN
        where_mat = tf.math.is_nan(x)
        
        #removes the NAN values from x and reshapes it into a 1D tensor. Note that -1 will automatically calculate the size of the tensor, and use it to reshape
        y = tf.reshape(x[~where_mat], (1, -1, 1))
        #tf.shape(y) returns the shape of y as a tensor, and [1] refers to the second dimension of the tensor
        self.dim1 = tf.shape(y)[1]

        return y

    def compute_output_shape(self, input_shape):
        #output shape is a tuple containing a single element, which is the size of the second dimension of the input tensor
        return [(1, self.dim1, 1)]
    


Let x be the input matrix:


```math
x = \begin{bmatrix}
1.0 & \text{NaN} & 3.0 & \text{NaN} & 5.0 & 7.0
\end{bmatrix}

```


The output matrix is then:

```math
 y = \begin{bmatrix}
\begin{bmatrix} 
1.0 \\ 
3.0 \\ 
5.0 \\ 
7.0 
\end{bmatrix}
\end{bmatrix}
```

#### Example usage

In [36]:
# Create an instance of the Vectorize class
vectorize_layer = Vectorize()

# Define an input tensor with some NaN values
x = tf.constant([1.0, float('nan'), 3.0, float('nan'), 5.0, 7.0], dtype=tf.float32)
print(f'the input tensor is {x}')

# Call the vectorize layer on the input tensor
result = vectorize_layer(x)


# Print the results
print(f'The output tensor is {result.numpy()}')




the input tensor is [ 1. nan  3. nan  5.  7.]
The output tensor is [[[1.]
  [3.]
  [5.]
  [7.]]]


## 3.Matrixation

In [37]:


# %% Creating matrixation layer
class Matrixize(Layer):
    """
    Layer that matrixizes the second dimension of inputs.
    
    init initialises the class. note that kwargs is a dictionary of keyword arguments.
    -N is the number of individuals (or countries in this case) 
    -T is the number of time steps
    -noObs is the number of observations
    -mask is a boolean matrix of shape (N, T) where True values indicate missing observations
    
    """

    def __init__(self, N, T, noObs, mask, **kwargs):
        super(Matrixize, self).__init__(**kwargs)
        self.N = N
        self.T = T
        self.noObs = noObs
        self.mask = mask

    def call(self, x):
        #identifies the indices of the False values in the mask. That is the indices of the non-missing observations
        where = ~self.mask
        
        #returns a tensor containing the indices of the non missing observations
        indices = tf.cast(tf.where(where), tf.int32)
        scatter = tf.scatter_nd(indices, tf.reshape(x, (-1,)), shape=tf.shape(self.mask))
        scatter = tf.cast(scatter, dtype=np.float64)

        indices = tf.cast(tf.where(~where), tf.int32)
        x_nan = tf.ones(self.N * self.T - self.noObs) * np.nan
        scatter_nan = tf.scatter_nd(indices, x_nan, shape=tf.shape(self.mask))
        scatter_nan = tf.cast(scatter_nan, dtype=np.float64)

        return scatter + scatter_nan

    def compute_output_shape(self, input_shape):
        return [(1, self.T, self.N)]



### Example usage of class

The input Tensor is the matrix: 

```math
\text{observations} = \begin{bmatrix}
1.0 & 2.0 & 3.0 & 4.0 & 5.0 & 6.0 & 7.0 & 8.0
\end{bmatrix}
```

The matrix of missing observations is: 
```math
\text{mask} = \begin{bmatrix}
\text{False} & \text{True}  & \text{False} & \text{True}  \\
\text{False} & \text{False} & \text{True}  & \text{False} \\
\text{True}  & \text{False} & \text{False} & \text{False}
\end{bmatrix}
```

```math
\text{output} = \begin{bmatrix}
1.0 & \text{NaN} & 2.0 & \text{NaN} \\
3.0 & 4.0 & \text{NaN} & 5.0 \\
\text{NaN} & 6.0 & 7.0 & 8.0
\end{bmatrix}

```

In [38]:

# Example Parameters
N = 3
T = 4
noObs = 8
mask = np.array([[False, True, False, True],
                 [False, False, True, False],
                 [True, False, False, False]])

# Input observations for the non-missing entries
observations = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0])

# Create the Matrixize layer
matrixize_layer = Matrixize(N, T, noObs, mask)

# Call the layer with the input observations
output = matrixize_layer(observations)

# Print the resulting matrix
print("Reconstructed Matrix:")
print(output.numpy())

print("Output Shape:")
print(matrixize_layer.compute_output_shape(observations.shape))


Reconstructed Matrix:
[[ 1. nan  2. nan]
 [ 3.  4. nan  5.]
 [nan  6.  7.  8.]]
Output Shape:
[(1, 4, 3)]


## 4.Extend

Very similar to Matricise. Maybe delete ?

In [39]:

# %% Creating extend layer
class Extend(Layer):
    """
    Layer that extends the second dimension of inputs.
    """

    def __init__(self, mask, **kwargs):
        super(Extend, self).__init__(**kwargs)
        self.mask = mask

    def call(self, x):
        where = ~self.mask
        indices = tf.cast(tf.where(where), tf.int32)
        scatter = tf.scatter_nd(indices, tf.reshape(x, (-1,)), shape=tf.shape(self.mask))
        scatter = tf.cast(scatter, dtype=np.float64)

        indices = tf.cast(tf.where(~where), tf.int32)
        mask_tmp = tf.cast(self.mask, tf.int32)
        x_nan = tf.ones(tf.reduce_sum(mask_tmp)) * np.nan
        scatter_nan = tf.scatter_nd(indices, x_nan, shape=tf.shape(self.mask))
        scatter_nan = tf.cast(scatter_nan, dtype=np.float64)

        return scatter + scatter_nan

    def compute_output_shape(self, input_shape):
        return [(1, self.T, 1)]



In [40]:

# %% Creating dummy layer
class Dummies(Layer):
    """
    Layer that creates country and time dummies.
    """

    def __init__(self, N, T, time_periods_na, **kwargs):
        super(Dummies, self).__init__(**kwargs)
        self.N = N
        self.T = T
        self.time_periods_na = time_periods_na
        self.noObs = None

    def call(self, x):
        where_mat = tf.transpose(tf.math.is_nan(x))

        for t in range(self.T):
            idx = tf.where(~where_mat[:, t, 0])
            idx = tf.reshape(idx, (-1,))

            D_t = tf.eye(self.N)
            D_t = tf.gather(D_t, idx, axis=0)

            if t == 0:
                Delta_1 = D_t

                Delta_2 = tf.matmul(D_t, tf.ones((self.N, 1)))

            else:
                Delta_1 = tf.concat([Delta_1, D_t], axis=0)

                Delta_2 = tf.concat([Delta_2, tf.zeros((tf.shape(Delta_2)[0], 1))], axis=1)

                Delta_2_tmp = tf.matmul(D_t, tf.ones((self.N, 1)))
                Delta_2_tmp = tf.concat([tf.zeros((tf.shape(Delta_2_tmp)[0], t)), Delta_2_tmp], axis=1)

                Delta_2 = tf.concat([Delta_2, Delta_2_tmp], axis=0)

        Delta_1 = Delta_1[:, 1:]
        Delta_2 = Delta_2[:, self.time_periods_na + 1:]

        self.noObs = tf.shape(Delta_1)[0]

        Delta_1 = tf.reshape(Delta_1, (1, self.noObs, self.N - 1))
        Delta_2 = tf.reshape(Delta_2, (1, self.noObs, self.T - (self.time_periods_na + 1)))

        return [Delta_1, Delta_2]

    def compute_output_shape(self, input_shape):
        return [(1, self.noObs, self.N - 1), (1, self.noObs, self.T - (self.time_periods_na + 1))]




### Example usage 

In [ ]:
# Input matrix x with NaN values
N = 4  # Number of countries
T = 3  # Number of time periods
time_periods_na = 1  # Number of missing time periods

x = tf.constant([
    [1.0, np.nan, 3.0],
    [2.0, 4.0, 5.0],
    [np.nan, 6.0, 7.0],
    [8.0, 9.0, np.nan]
], dtype=tf.float32)

# Initialize and apply the Dummies layer
dummies_layer = Dummies(N=N, T=T, time_periods_na=time_periods_na)
Delta_1, Delta_2 = dummies_layer(x)

# Print the results
print("Delta_1 (Country Dummies):")
print(Delta_1.numpy())

print("\nDelta_2 (Time Dummies):")
print(Delta_2.numpy())

In [29]:

def individual_loss(mask):
    """
    Loss function (in two layers so that it can be interpreted by tensorflow).

    ARGUMENTS
        * mask: mask used to identify missing observations.

    Returns
        * loss: loss function evaluated in y_true and y_pred.
    """

    def loss(y_true, y_pred):
        """
        ARGUMENTS
            * y_true: observed targets.
            * y_pred: predicted targets.

        RETURNS
            * loss function evaluated in y_true and y_pred.
        """

        y_true_transf = tf.reshape(y_true[~mask], (1, -1, 1))
        y_pred_transf = tf.reshape(y_pred[~mask], (1, -1, 1))

        return tf.reduce_mean(tf.math.squared_difference(y_true_transf, y_pred_transf), axis=1)

    return loss
